In [1]:
# Import statements
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import folium

key = '7260c924edcb41fc86fd68d3f0c96c3b'


# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

# making the 1st column, index
df_product_a = pd.read_csv("product_a.csv", index_col=0)

# Generating the Dataframe with Coordinates

In [2]:
dfL = df_product_a.groupby('location').aggregate('sum')
#dfLocation.rename(columns={ dfLocation.columns[0]: "City" }, inplace = True)
del dfL['year']
dfL=dfL.round(3)
dfLocation = dfL.reset_index()

geocoder = OpenCageGeocode(key)
for index, row in dfLocation.iterrows():
    City = row['location']      
    query = str(City)
    results = geocoder.geocode(query)
    try:
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
    except IndexError:
        dfLocation.at[index, 'location'] = "New York"

dfM = dfLocation.groupby('location').aggregate('sum')
dfMap = dfM.reset_index()

list_lat = []   # create empty lists
list_long = []

for index, row in dfMap.iterrows(): # iterate over rows in dataframe

    City = row['location']
           
    query = str(City)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)

# create new columns from lists    

dfMap['lat'] = list_lat   
dfMap['lon'] = list_long


# Generating the Folium Map

In [3]:
def colo_r(bt): 
    if int(bt) in range(0,400): 
        col = 'green'
    elif int(bt) in range(451,500): 
        col = 'blue'
    elif int(bt) in range(501,600): 
        col = 'orange'
    else: 
        col='red'
    return col

m = folium.Map(location=[31.51073, -96.4247], zoom_start=5)

for _, row in dfMap.iterrows():
    folium.Marker(
    location=[row['lat'], row['lon']],
    popup= "bags_t : "+ str(row['bags_t']),
    icon=folium.Icon(color=colo_r(row['price']),tooltip=row['location'], icon='info-sign')
    ).add_to(m)

m